In [14]:
from pyspark.sql import SparkSession

# Create a Spark session and context (RDD and core functionality)
# uses 2 cores, you can also local[*] for all cores
spark = SparkSession.builder.master('local[2]').getOrCreate()
sc = spark.sparkContext

In [5]:
# Sum of the first 1 bilion of whole numbers

# This builds an RDD which is Spark's most basic distributed data structure
# You can think about it as a collection of records distributed accross a cluster
# It's very raw, in the sense that it does not know the concept of rows or columns
# Good when you want to work with unstructured data
rdd = sc.parallelize(range(1000000000 + 1))

# This will mean that several records will get disributed through the cluster into n PARTITIONS

# Sum all the numbers in that collection
rdd.sum()

# see how 2 cores processes data faster than 1 core (50% of the time)
# Same would likely happen in a cluster of 1 executor or 2 executors.
# Let's go see what is happening in the Spark UI: localhost:4040

500000000500000000

In [6]:
# What about transforming the data?
rdd.map(lambda num: num * 2)

# Each node will process their partitions, therefore they will have n TASKS to do
# Each tasks will be about taking their portion of the data and, in this case, multiply the numbers by 2
# Each cluster node will intervene in their PARTITIONS only, not the entire dataset

# What happens if you execute this now? Hmm... Another RDD? Strange

PythonRDD[2] at RDD at PythonRDD.scala:53

In [16]:
# Spark will return an RDD after each transformation, so let's save it
# Note, let's work with less data because transforming 1 billion records is taxing
rdd = sc.parallelize(range(1000000 + 1))
double_rdd = rdd.map(lambda num: num * 10)

# Remember map and reduce? Well, they're back

# Count number of records
#print(double_rdd.reduce(lambda a, b: a + b))

# which is the same as this
#print(double_rdd.sum())

# Actions trigger Spark to perform the computation
# We call Spark Lazy because until there's an action on the data, there's no distributed computation happening
# Transformations (which return an RDD each) will be chained
# Only when Spark see's an action it will trigger the computation
# We call this lazy evaluation
# An action can be:
# count, sum
# write (to write to a storage system like HDFS)
# persist to store in memory
# take or collect to take sample or collect RDD to the Spark driver

In [11]:
#Driver what?

#This executes in the Driver... just a regular python app, not distributed, therefore not scalable
print(1 + 1)

#This executes in the Spark Executors (Workers), therefore in a cluster, therefore scalable
print(double_rdd.count())

#Want to bring Spark RDD into the Driver to maybe visualize with Matplotlib or execute regular python code?

# take a sample
print(double_rdd.take(10))

# well, this crashes
print(double_rdd.collect())


# Do not send Big Data into the driver
# Big Data should be processed in Spark executors only
# Spark will take care of it for you, only send the aggregated/filtered results into the driver

2
1000001
[0, 10, 20, 30, 40, 50, 60, 70, 80, 90]


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [12]:
# Stop Spark Session
spark.stop()

In [ ]:
# More about SPARK RDDs can be found here: https://spark.apache.org/docs/latest/rdd-programming-guide.html#transformations
# For now we will focus more on Dataframes and Datasets